## 1. Предобработка данных

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats as st

In [64]:
# address_client = pd.read_csv(r'C:\WORK\STUDY\GIT\Данные для хакатона\address_client.csv', encoding = "utf-8", on_bad_lines='skip', 
# names=['id_client_rf', 'address', 'flag_provider', 'nflag_have_free_ports', 'tech_name', 'latitude', 'longitude'])

# base_info_client = pd.read_csv(r'C:\WORK\STUDY\GIT\Данные для хакатона\base_info_client.csv', encoding = "utf-8", 
# names=['id_client_rf', 'name_rf', 'id_client', 'workersrange', 'total_revenue', 'companywithsameinfo', 'arbitrationcases',
# 'datefirstreg', 'flag_site', 'flag_business_center', 'flag_resident', 'date_actual_data', 'flag_vk'], parse_dates=['datefirstreg','date_actual_data'])

# final_list = pd.read_csv(r'C:\WORK\STUDY\GIT\Данные для хакатона\final_list.csv', encoding = "utf-8", names=['id_client_rf'])

# okved_industry_client = pd.read_csv(r'C:\WORK\STUDY\GIT\Данные для хакатона\okved_industry_client.csv', encoding = "utf-8", 
# names = ['id_client_rf', 'Code_okved', 'Name_okved', 'Type_okved', 'Industry'])

sales = pd.read_excel(r'C:\WORK\STUDY\GIT\Данные для хакатона\sales.xlsx',
names = ['month_sale', 'day_sale', 'id_client_rf', 'sale_size'])

In [56]:
pd.set_option('display.max_columns', 50)
print('Размерность address_client:', address_client.shape[0])
print(address_client.info())
print('Размерность base_info_client:', base_info_client.shape[0])
print(base_info_client.info())
print('Размерность final_list:', final_list.shape[0])
print(final_list.info())
print('Размерность okved_industry_client:', okved_industry_client.shape[0])
print(okved_industry_client.info())
print('Размерность sales:', sales.shape[0])
print(sales.info())

Размерность address_client: 1485674
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1485674 entries, 0 to 1485673
Data columns (total 7 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   id_client_rf           1485674 non-null  object 
 1   address                1485674 non-null  object 
 2   flag_provider          1485674 non-null  int64  
 3   nflag_have_free_ports  1485674 non-null  int64  
 4   tech_name              736940 non-null   object 
 5   latitude               1259978 non-null  float64
 6   longitude              1259978 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 79.3+ MB
None
Размерность base_info_client: 2218328
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2218328 entries, 0 to 2218327
Data columns (total 13 columns):
 #   Column                Dtype         
---  ------                -----         
 0   id_client_rf          int64         
 1   name_rf         

In [39]:
# Проверка кол-ва уникальных значений для столбцов, где должно быть булево значение
print('address_client:\n', address_client.nunique(), sep='')
print()
print('base_info_client:\n', base_info_client.nunique(), sep='')
print()
print('final_list:\n', final_list.nunique(), sep='')
print()
print('okved_industry_client:\n', okved_industry_client.nunique(), sep='')
print()
print('sales:\n', sales.nunique(), sep='')

address_client:
id_client_rf             587676
address                  183457
flag_provider                 2
nflag_have_free_ports         2
tech_name                    40
latitude                 114296
longitude                116575
dtype: int64

base_info_client:
id_client_rf            2218328
name_rf                      13
id_client               2218328
workersrange                 11
total_revenue              1559
companywithsameinfo         135
arbitrationcases            575
datefirstreg              12865
flag_site                     2
flag_business_center          2
flag_resident                 2
date_actual_data           4551
flag_vk                       2
dtype: int64

final_list:
id_client_rf    1950766
dtype: int64

okved_industry_client:
id_client_rf    2139266
Code_okved         2470
Name_okved         2396
Type_okved            2
Industry             33
dtype: int64

sales:
month_sale         7
day_sale         193
id_client_rf    9316
sale_size          3


In [18]:
# 1. address_client
address_client.head()

,id_client_rf,address,flag_provider,nflag_have_free_ports,tech_name,latitude,longitude
0,00840127_Ф13,"уфа г.\, лесотехникума улица\, 49/1",1,0,NaN,NaN,NaN
1,00840181_Ф13,"уфа г.\, лесотехникума улица\, 49/1",1,0,NaN,NaN,NaN
2,00839528_Ф13,"уфа г.\, лесотехникума улица\, 49/1",1,0,NaN,NaN,NaN
3,00839607_Ф13,"уфа г.\, лесотехникума улица\, 49/1",1,0,NaN,NaN,NaN
4,00075449_Ф13,"уфа г.\, лесотехникума улица\, 49/1",1,0,NaN,NaN,NaN


In [77]:
# 2. base_info_client
base_info_client.head()

,id_client_rf,name_rf,id_client,workersrange,total_revenue,companywithsameinfo,arbitrationcases,datefirstreg,flag_site,flag_business_center,flag_resident,date_actual_data,flag_vk
0,1631354_Ф09,Ф09,01631354_Ф09,NaN,0,1.0,NaN,2020-01-29,0,0,0,NaT,0
1,1631368_Ф09,Ф09,01631368_Ф09,NaN,0,1.0,NaN,2020-07-31,0,0,0,NaT,0
2,1631377_Ф09,Ф09,01631377_Ф09,NaN,0,1.0,NaN,2020-08-17,0,0,0,NaT,0
3,1631387_Ф09,Ф09,01631387_Ф09,NaN,0,1.0,NaN,2019-12-23,0,0,0,NaT,0
4,1631409_Ф09,Ф09,01631409_Ф09,NaN,0,1.0,NaN,2020-10-19,0,0,0,NaT,0


In [76]:
# Смотрим в каком формате хранятся записи workersrange. Нужно будет оставить только правую часть.
# base_info_client[base_info_client['workersrange'].notnull()==True]
# Переводим workersrange в числовой формат по правой цифре. 
base_info_client['workersrange'] = base_info_client['workersrange'].str.replace('(.* \.\. )|(.*\.\.)','', regex=True).dropna().astype(int)
# Переводим id_client_rf в такой же формат, как и в других таблицах
base_info_client['id_client_rf'] = base_info_client['id_client_rf'].astype(str)+'_'+base_info_client['name_rf'].astype(str)

In [20]:
# 3. final_list
final_list.head()

,id_client_rf
0,01768987_Ф04
1,01769012_Ф04
2,01769016_Ф04
3,01769018_Ф04
4,01769044_Ф04


In [21]:
# 4. okved_industry_client
okved_industry_client.head()

,id_client_rf,Code_okved,Name_okved,Type_okved,Industry
0,00811390_Ф13,47.91.2,"Торговля розничная\, осуществляемая непосредст...",1,Интернет-бизнес
1,01516426_Ф10,47.91.2,"Торговля розничная\, осуществляемая непосредст...",1,Интернет-бизнес
2,01258497_Ф06,47.91.2,"Торговля розничная\, осуществляемая непосредст...",1,Интернет-бизнес
3,01583065_Ф12,47.91.2,"Торговля розничная\, осуществляемая непосредст...",1,Интернет-бизнес
4,01506195_Ф04,47.91.2,"Торговля розничная\, осуществляемая непосредст...",1,Интернет-бизнес


In [74]:
# 5. sales
sales.head()
# sales['sale_size'].unique()

,month_sale,day_sale,id_client_rf,sale_size
0,2023-03-01,2023-03-16,00309682_Ф12,3
1,2023-03-01,2023-03-16,00309999_Ф12,3
2,2023-03-01,2023-03-07,00333129_Ф12,3
3,2023-03-01,2023-03-24,00333306_Ф12,3
4,2023-03-01,2023-03-07,00333741_Ф12,3


In [73]:
# Переводим sale_size в цифровой формат
sales['sale_size'] = np.where(sales['sale_size']=='High     ', 3,
 np.where(sales['sale_size']=='Middle   ', 2,
 np.where(sales['sale_size']=='Low      ', 1, 0)))

In [38]:
# Анализ пропусков
print('address_client:\n', address_client.isna().sum(), sep='')
print()
print('base_info_client:\n', base_info_client.isna().sum(), sep='')
print()
print('final_list:\n', final_list.isna().sum(), sep='')
print()
print('okved_industry_client:\n', okved_industry_client.isna().sum(), sep='')
print()
print('sales:\n', sales.isna().sum(), sep='')


address_client:
id_client_rf                  0
address                       0
flag_provider                 0
nflag_have_free_ports         0
tech_name                748734
latitude                 225696
longitude                225696
dtype: int64

base_info_client:
id_client_rf                  0
name_rf                       0
id_client                     0
workersrange            2216478
total_revenue                 0
companywithsameinfo     1321582
arbitrationcases        1781086
datefirstreg              10800
flag_site                     0
flag_business_center          0
flag_resident                 0
date_actual_data        1751900
flag_vk                       0
dtype: int64

final_list:
id_client_rf    0
dtype: int64

okved_industry_client:
id_client_rf      0
Code_okved        0
Name_okved        0
Type_okved        0
Industry        236
dtype: int64

sales:
month_sale      0
day_sale        0
id_client_rf    0
sale_size       0
dtype: int64


In [40]:
# Анализ дубликатов
print('address_client:\n', address_client.duplicated().sum())
print()
print('base_info_client:\n', base_info_client.duplicated().sum())
print()
print('final_list:\n', final_list.duplicated().sum())
print()
print('okved_industry_client:\n', okved_industry_client.duplicated().sum())
print()
print('sales:\n', sales.duplicated().sum())

address_client:
 0

base_info_client:
 0

final_list:
 0

okved_industry_client:
 0

sales:
 0


In [92]:
# Создаем единый датасет (не уверен, что правильно смержил)
# .merge(base_info_client, how='left', on='id_client_rf' ).merge(address_client, how='left', on='id_client_rf' )
df_union = sales.merge(okved_industry_client, how='left', on='id_client_rf' ).merge(base_info_client, how='left', on='id_client_rf' ).merge(address_client, how='left', on='id_client_rf' )

In [93]:
df_union.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5581374 entries, 0 to 5581373
Data columns (total 8 columns):
 #   Column        Dtype         
---  ------        -----         
 0   month_sale    datetime64[ns]
 1   day_sale      datetime64[ns]
 2   id_client_rf  object        
 3   sale_size     float64       
 4   Code_okved    object        
 5   Name_okved    object        
 6   Type_okved    int64         
 7   Industry      object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(4)
memory usage: 340.7+ MB


In [80]:
df_union

,id_client_rf,name_rf,id_client,workersrange,total_revenue,companywithsameinfo,arbitrationcases,datefirstreg,flag_site,flag_business_center,flag_resident,date_actual_data,flag_vk,Code_okved,Name_okved,Type_okved,Industry
0,1631354_Ф09,Ф09,01631354_Ф09,NaN,0,1.0,NaN,2020-01-29,0,0,0,NaT,0,NaN,NaN,NaN,NaN
1,1631368_Ф09,Ф09,01631368_Ф09,NaN,0,1.0,NaN,2020-07-31,0,0,0,NaT,0,NaN,NaN,NaN,NaN
2,1631377_Ф09,Ф09,01631377_Ф09,NaN,0,1.0,NaN,2020-08-17,0,0,0,NaT,0,NaN,NaN,NaN,NaN
3,1631387_Ф09,Ф09,01631387_Ф09,NaN,0,1.0,NaN,2019-12-23,0,0,0,NaT,0,NaN,NaN,NaN,NaN
4,1631409_Ф09,Ф09,01631409_Ф09,NaN,0,1.0,NaN,2020-10-19,0,0,0,NaT,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2218323,1067702_Ф05,Ф05,01067702_Ф05,NaN,0,NaN,NaN,2022-06-14,0,0,0,NaT,0,NaN,NaN,NaN,NaN
2218324,1053763_Ф05,Ф05,01053763_Ф05,NaN,0,NaN,NaN,2022-04-04,0,0,0,NaT,0,NaN,NaN,NaN,NaN
2218325,1053883_Ф05,Ф05,01053883_Ф05,NaN,0,NaN,NaN,2022-04-04,0,0,0,NaT,0,NaN,NaN,NaN,NaN
2218326,1054804_Ф05,Ф05,01054804_Ф05,NaN,0,NaN,NaN,2022-06-14,0,0,0,NaT,0,NaN,NaN,NaN,NaN
